# Wrangle and Analyze Data

## Table of Contents
- [Gather](#gather)
- [Assess](#assess)
- [Clean](#clean)

In [1]:
import pandas as pd
import numpy as np
import requests
import os
import tweepy
import json
import config

<a id='gather'></a>
<h2 id="-Gather" style="
    background-color: #555;
    color: #eee;
    padding: 10px 5px;
">Gather</h2>

In [2]:
df_twitter_archive_enhanced = pd.read_csv('twitter-archive-enhanced.csv')

In [3]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

with open(os.path.join(os.getcwd(),'image-predictions.tsv'), mode='wb') as file:
    file.write(response.content)

df_image_predictions = pd.read_csv('image-predictions.tsv', sep='\t')

In [4]:
auth = tweepy.OAuthHandler(config.consumer_key, config.consumer_secret)
auth.set_access_token(config.access_token, config.access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [5]:
tweet_id_list = pd.concat( [df_image_predictions['tweet_id'], 
                          df_twitter_archive_enhanced['tweet_id']], ignore_index=True)

tweet_id_list = tweet_id_list.drop_duplicates()

In [6]:
# this function just to calculate the performance 
time_prv_step = time_step = None
def performance(count,iterations):
    progress = round((count/iterations)*100,2)
    print('processing {}%'.format(progress), end='\r')
    if(count%round(iterations/10) == 0): 
        global time_prv_step
        global time_step
        
        time_prv_step = pd.Timestamp.now() if count == 0 else time_step
        time_step = pd.Timestamp.now()
        
        duration = (time_step - time_prv_step).total_seconds()
        print('{} samples, during {}s'.format(count, round(duration,2) ))
        time_start = time_step

In [31]:
tweets = []
tweet_id_list_error = []
iterations = tweet_id_list.shape[0]
count = 0
for tweet_id in tweet_id_list:
    performance(count, iterations)
    count += 1
    try:
        tweets.append(api.get_status(tweet_id)._json)
    except Exception as e:
        print(str(tweet_id) + ": " + str(e))
        tweet_id_list_error.append(tweet_id)

df_tweets = pd.DataFrame(df_tweets)
df_tweets.to_csv('tweets.csv', index=False)

0 samples, during 0.0s
236 samples, during 237.16s
472 samples, during 216.98s
680055455951884288: [{'code': 144, 'message': 'No status found with that ID.'}]
708 samples, during 209.0s


Rate limit reached. Sleeping for: 77


944 samples, during 285.19s
1180 samples, during 212.78s
754011816964026368: [{'code': 144, 'message': 'No status found with that ID.'}]
1416 samples, during 214.6s
802247111496568832: [{'code': 144, 'message': 'No status found with that ID.'}]
1652 samples, during 201.68s
829374341691346946: [{'code': 144, 'message': 'No status found with that ID.'}]


Rate limit reached. Sleeping for: 102


837012587749474308: [{'code': 144, 'message': 'No status found with that ID.'}]
837366284874571778: [{'code': 144, 'message': 'No status found with that ID.'}]
842892208864923648: [{'code': 144, 'message': 'No status found with that ID.'}]
844704788403113984: [{'code': 144, 'message': 'No status found with that ID.'}]
1888 samples, during 319.52s
851953902622658560: [{'code': 144, 'message': 'No status found with that ID.'}]
861769973181624320: [{'code': 144, 'message': 'No status found with that ID.'}]
872261713294495745: [{'code': 144, 'message': 'No status found with that ID.'}]
873697596434513921: [{'code': 144, 'message': 'No status found with that ID.'}]
888202515573088257: [{'code': 144, 'message': 'No status found with that ID.'}]
872668790621863937: [{'code': 144, 'message': 'No status found with that ID.'}]
869988702071779329: [{'code': 144, 'message': 'No status found with that ID.'}]
866816280283807744: [{'code': 144, 'message': 'No status found with that ID.'}]
85660299358

In [9]:
df_tweets = pd.read_csv('tweets.csv')

> There are 22 tweets that no longer exist.

<a id='assess'></a>
<h2 id="-Gather" style="
    background-color: #555;
    color: #eee;
    padding: 10px 5px;
">Assess</h2>

<h3 id="-Gather" style="
    background-color: #bbb;
    color: #fff;
    padding: 10px 5px;
">twitter_archive_enhanced table</h3>

In [11]:
df_twitter_archive_enhanced.sample(10)

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
204   852936405516943360                    NaN                  NaN   
2105  670474236058800128                    NaN                  NaN   
2212  668587383441514497                    NaN                  NaN   
2009  672254177670729728                    NaN                  NaN   
2198  668815180734689280                    NaN                  NaN   
917   756939218950160384                    NaN                  NaN   
713   784183165795655680                    NaN                  NaN   
289   838201503651401729                    NaN                  NaN   
523   809448704142938112                    NaN                  NaN   
1039  744234799360020481                    NaN                  NaN   

                      timestamp  \
204   2017-04-14 17:27:40 +0000   
2105  2015-11-28 05:28:09 +0000   
2212  2015-11-23 00:30:28 +0000   
2009  2015-12-03 03:21:00 +0000   
2198  2015-11-23 15:35:39 +0000   
917   2016-07-23 19:49:07 +0000   
713   2016-10-07 00:06:50 +0000   
289   2017-03-05 01:36:26 +0000   
523   2016-12-15 17:23:04 +0000   
1039  2016-06-18 18:26:18 +0000   

                                                 source  \
204   <a href="http://twitter.com/download/iphone" r...   
2105  <a href="http://twitter.com/download/iphone" r...   
2212  <a href="http://vine.co" rel="nofollow">Vine -...   
2009  <a href="http://twitter.com/download/iphone" r...   
2198  <a href="http://twitter.com/download/iphone" r...   
917   <a href="http://twitter.com/download/iphone" r...   
713   <a href="http://vine.co" rel="nofollow">Vine -...   
289   <a href="http://twitter.com/download/iphone" r...   
523   <a href="http://twitter.com/download/iphone" r...   
1039  <a href="http://twitter.com/download/iphone" r...   

                                                   text  retweeted_status_id  \
204   RT @dog_rates: I usually only share these on F...         8.316501e+17   
2105  Honor to rate this dog. Great teeth. Nice horn...                  NaN   
2212  Never forget this vine. You will not stop watc...                  NaN   
2009  This is Rolf. He's having the time of his life...                  NaN   
2198  This is a wild Toblerone from Papua New Guinea...                  NaN   
917   This is Jax. He is a majestic mountain pupper....                  NaN   
713   This is Reginald. He's one magical puppo. Aero...                  NaN   
289   RT @dog_rates: Meet Sunny. He can take down a ...         8.207497e+17   
523   I call this one "A Blep by the Sea" 12/10 http...                  NaN   
1039  Here's a doggo realizing you can stand in a po...                  NaN   

      retweeted_status_user_id retweeted_status_timestamp  \
204               4.196984e+09  2017-02-14 23:43:18 +0000   
2105                       NaN                        NaN   
2212                       NaN                        NaN   
2009                       NaN                        NaN   
2198                       NaN                        NaN   
917                        NaN                        NaN   
713                        NaN                        NaN   
289               4.196984e+09  2017-01-15 21:49:15 +0000   
523                        NaN                        NaN   
1039                       NaN                        NaN   

                                          expanded_urls  rating_numerator  \
204   http://www.gofundme.com/bluethewhitehusky,http...                13   
2105  https://twitter.com/dog_rates/status/670474236...                10   
2212                      https://vine.co/v/ea0OwvPTx9l                13   
2009  https://twitter.com/dog_rates/status/672254177...                11   
2198  https://twitter.com/dog_rates/status/668815180...                 7   
917   https://twitter.com/dog_rates/status/756939218...                12   
713                       https://vine.co/v/5ghHLBMMdlV                12   
289   https://twit

In [55]:
df_twitter_archive_enhanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [57]:
df_twitter_archive_enhanced.describe()

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
count  2.356000e+03           7.800000e+01         7.800000e+01   
mean   7.427716e+17           7.455079e+17         2.014171e+16   
std    6.856705e+16           7.582492e+16         1.252797e+17   
min    6.660209e+17           6.658147e+17         1.185634e+07   
25%    6.783989e+17           6.757419e+17         3.086374e+08   
50%    7.196279e+17           7.038708e+17         4.196984e+09   
75%    7.993373e+17           8.257804e+17         4.196984e+09   
max    8.924206e+17           8.862664e+17         8.405479e+17   

       retweeted_status_id  retweeted_status_user_id  rating_numerator  \
count         1.810000e+02              1.810000e+02       2356.000000   
mean          7.720400e+17              1.241698e+16         13.126486   
std           6.236928e+16              9.599254e+16         45.876648   
min           6.661041e+17              7.832140e+05          0.000000   
25%           7.186315e+17              4.196984e+09         10.000000   
50%           7.804657e+17              4.196984e+09         11.000000   
75%           8.203146e+17              4.196984e+09         12.000000   
max           8.874740e+17              7.874618e+17       1776.000000   

       rating_denominator  
count         2356.000000  
mean            10.455433  
std              6.745237  
min              0.000000  
25%             10.000000  
50%             10.000000  
75%             10.000000  
max            170.000000

In [25]:
for col in df_twitter_archive_enhanced.columns:
    print(df_twitter_archive_enhanced[col].value_counts().count(), '\t', col)

2356 	 tweet_id
77 	 in_reply_to_status_id
31 	 in_reply_to_user_id
2356 	 timestamp
4 	 source
2356 	 text
181 	 retweeted_status_id
25 	 retweeted_status_user_id
181 	 retweeted_status_timestamp
2218 	 expanded_urls
40 	 rating_numerator
18 	 rating_denominator
957 	 name
2 	 doggo
2 	 floofer
2 	 pupper
2 	 puppo


In [51]:
df_twitter_archive_enhanced.rating_numerator.value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

In [50]:
df_twitter_archive_enhanced.rating_denominator.value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

In [135]:
df_twitter_archive_enhanced.text.str.contains(r'\d+\/\d+').value_counts()

True    2356
Name: text, dtype: int64

In [134]:
df_twitter_archive_enhanced.text.str.extract(r'(?P<text>\d+\/\d+)').text.value_counts()

12/10      558
11/10      463
10/10      461
13/10      351
9/10       156
8/10       102
14/10       54
7/10        53
5/10        37
6/10        32
3/10        19
4/10        15
2/10         9
1/10         8
4/20         2
9/11         2
75/10        2
420/10       2
15/10        2
0/10         2
45/50        1
144/120      1
17/10        1
666/10       1
7/11         1
60/50        1
1776/10      1
84/70        1
88/80        1
143/130      1
121/110      1
182/10       1
11/15        1
20/16        1
50/50        1
204/170      1
99/90        1
1/2          1
26/10        1
007/10       1
960/00       1
165/150      1
27/10        1
80/80        1
44/40        1
24/7         1
Name: text, dtype: int64

In [144]:
# retweets in tweets
df_twitter_archive_enhanced[
    df_twitter_archive_enhanced.retweeted_status_id.isin(df_twitter_archive_enhanced.tweet_id)
].shape[0]


156

In [145]:
# replies in tweets
df_twitter_archive_enhanced[
    df_twitter_archive_enhanced.in_reply_to_status_id.isin(df_twitter_archive_enhanced.tweet_id)
].shape[0]

44

<h3 id="-Gather" style="
    background-color: #bbb;
    color: #fff;
    padding: 10px 5px;
">image_predictions table</h3>

In [54]:
df_image_predictions.sample(20)

tweet_id                                            jpg_url  \
972   706644897839910912  https://pbs.twimg.com/ext_tw_video_thumb/70664...   
1397  768193404517830656    https://pbs.twimg.com/media/Cqkr0wiW8AAn2Oi.jpg   
1189  739606147276148736    https://pbs.twimg.com/media/CkOb3FXW0AAUL_U.jpg   
1572  795076730285391872    https://pbs.twimg.com/media/CwiuEJmW8AAZnit.jpg   
1263  749036806121881602    https://pbs.twimg.com/media/CmUciKgWIAA97sH.jpg   
931   703079050210877440    https://pbs.twimg.com/media/CcHWqQCW8AEb0ZH.jpg   
566   678334497360859136    https://pbs.twimg.com/media/CWntoDVWcAEl3NB.jpg   
1345  759159934323924993    https://pbs.twimg.com/media/CU1zsMSUAAAS0qW.jpg   
845   695095422348574720    https://pbs.twimg.com/media/CaV5mRDXEAAR8iG.jpg   
1945  862096992088072192    https://pbs.twimg.com/media/C_bIo7QXYAAGfPu.jpg   
1283  750429297815552001    https://pbs.twimg.com/media/CmoPdmHW8AAi8BI.jpg   
1792  830956169170665475  https://pbs.twimg.com/ext_tw_video_thumb/83095...   
1669  813066809284972545    https://pbs.twimg.com/media/C0iX8OOVEAEIpMC.jpg   
868   697596423848730625    https://pbs.twimg.com/media/Ca5cPrJXIAImHtD.jpg   
101   667793409583771648    https://pbs.twimg.com/media/CUR6jqVWsAEgGot.jpg   
286   671151324042559489    https://pbs.twimg.com/media/CVBokRSWsAADuXx.jpg   
518   676470639084101634    https://pbs.twimg.com/media/CWNOdIpWoAAWid2.jpg   
1302  752917284578922496    https://pbs.twimg.com/media/CnLmRiYXEAAO_8f.jpg   
1003  708834316713893888    https://pbs.twimg.com/media/CdZI_bpWEAAm1fs.jpg   
1584  797236660651966464    https://pbs.twimg.com/media/CxBafisWQAAtJ1X.jpg   

      img_num                        p1   p1_conf  p1_dog  \
972         1              space_heater  0.137871   False   
1397        1                      lion  0.396984   False   
1189        3          Blenheim_spaniel  0.933755    True   
1572        2                  gas_pump  0.676439   False   
1263        1  sulphur-crested_cockatoo  0.960276   False   
931         2                  Pembroke  0.778503    True   
566         1           Norfolk_terrier  0.378643    True   
1345        1             Irish_terrier  0.254856    True   
845         1                  papillon  0.227784    True   
1945        2                      chow  0.677589    True   
1283        1          golden_retriever  0.964929    True   
1792        1                    kuvasz  0.451516    True   
1669        1               toy_terrier  0.776400    True   
868         1         Shetland_sheepdog  0.621668    True   
101         1                 dalmatian  0.535073    True   
286         1                Rottweiler  0.781201    True   
518         1          golden_retriever  0.790386    True   
1302        1           German_shepherd  0.609283    True   
1003        1                Eskimo_dog  0.283945    True   
1584        2                    collie  0.767005    True   

                               p2   p2_conf  p2_dog  \
972                     Chihuahua  0.132928    True   
1397                          ram  0.300851   False   
1189               cocker_spaniel  0.041719    True   
1572                    harvester  0.049995   False   
1263  West_Highland_white_terrier  0.019522    True   
931             Shetland_sheepdog  0.093834    True   
566              golden_retriever  0.095594    True   
1345                       briard  0.227716    True   
845                     Chihuahua  0.218128    True   
1945                   Pomeranian  0.270648    True   
1283           Labrador_retriever  0.011584    True   
1792             golden_retriever  0.317196    True   
1669                     Pembroke  0.115034    True   
868                        collie  0.366578    True   
101                English_setter  0.451219    True   
286       black-and-tan_coonhound  0.061206    True   
518                        borzoi  0.022885    True   
1302                     malinois  0.352460    True   
1003                  giant_p

In [56]:
df_image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [194]:
df_image_predictions.describe()

tweet_id      img_num      p1_conf       p2_conf       p3_conf
count  2.075000e+03  2075.000000  2075.000000  2.075000e+03  2.075000e+03
mean   7.384514e+17     1.203855     0.594548  1.345886e-01  6.032417e-02
std    6.785203e+16     0.561875     0.271174  1.006657e-01  5.090593e-02
min    6.660209e+17     1.000000     0.044333  1.011300e-08  1.740170e-10
25%    6.764835e+17     1.000000     0.364412  5.388625e-02  1.622240e-02
50%    7.119988e+17     1.000000     0.588230  1.181810e-01  4.944380e-02
75%    7.932034e+17     1.000000     0.843855  1.955655e-01  9.180755e-02
max    8.924206e+17     4.000000     1.000000  4.880140e-01  2.734190e-01

In [59]:
for col in df_image_predictions.columns:
    print(df_image_predictions[col].value_counts().count(), '\t', col)

2075 	 tweet_id
2009 	 jpg_url
4 	 img_num
378 	 p1
2006 	 p1_conf
2 	 p1_dog
405 	 p2
2004 	 p2_conf
2 	 p2_dog
408 	 p3
2006 	 p3_conf
2 	 p3_dog


In [146]:
df_image_predictions.p1.value_counts()

golden_retriever             150
Labrador_retriever           100
Pembroke                      89
Chihuahua                     83
pug                           57
chow                          44
Samoyed                       43
toy_poodle                    39
Pomeranian                    38
malamute                      30
cocker_spaniel                30
French_bulldog                26
miniature_pinscher            23
Chesapeake_Bay_retriever      23
seat_belt                     22
Siberian_husky                20
Staffordshire_bullterrier     20
German_shepherd               20
Cardigan                      19
web_site                      19
Shetland_sheepdog             18
beagle                        18
teddy                         18
Eskimo_dog                    18
Maltese_dog                   18
Lakeland_terrier              17
Rottweiler                    17
Shih-Tzu                      17
kuvasz                        16
Italian_greyhound             16
          

In [152]:
df_image_predictions.p2.value_counts()

Labrador_retriever                104
golden_retriever                   92
Cardigan                           73
Chihuahua                          44
Pomeranian                         42
Chesapeake_Bay_retriever           41
French_bulldog                     41
toy_poodle                         37
cocker_spaniel                     34
Siberian_husky                     33
miniature_poodle                   33
beagle                             28
collie                             27
Eskimo_dog                         27
Pembroke                           27
kuvasz                             26
Italian_greyhound                  22
American_Staffordshire_terrier     21
Pekinese                           21
chow                               20
toy_terrier                        20
malinois                           20
Samoyed                            20
miniature_pinscher                 20
Norwegian_elkhound                 19
Boston_bull                        19
Staffordshir

In [153]:
df_image_predictions.p3.value_counts()

Labrador_retriever                79
Chihuahua                         58
golden_retriever                  48
Eskimo_dog                        38
kelpie                            35
kuvasz                            34
chow                              32
Staffordshire_bullterrier         32
cocker_spaniel                    31
beagle                            31
Pomeranian                        29
toy_poodle                        29
Pekinese                          29
Great_Pyrenees                    27
Pembroke                          27
Chesapeake_Bay_retriever          27
malamute                          26
French_bulldog                    26
American_Staffordshire_terrier    24
pug                               23
Cardigan                          23
basenji                           21
toy_terrier                       20
bull_mastiff                      20
Siberian_husky                    19
Shetland_sheepdog                 17
Boston_bull                       17
L

In [66]:
len(df_twitter_archive_enhanced)

2356

In [193]:
x = ['doggo', 'floofer', 'pupper', 'puppo']
y = pd.DataFrame(
    list(df_image_predictions.p1)+list(df_image_predictions.p2)+list(df_image_predictions.p3)
    ,columns=['type']
)

print(len(y))
print(len(y.drop_duplicates()))
print(y.drop_duplicates().isin(x).type.value_counts())

6225
634
False    634
Name: type, dtype: int64


In [201]:
df_image_predictions.query('p1_dog == False & p2_dog == False & p3_dog == False').shape[0]

324

<h3 id="-Gather" style="
    background-color: #bbb;
    color: #fff;
    padding: 10px 5px;
">tweets table</h3>

In [136]:
df_tweets.sample(10)

contributors  coordinates                      created_at  \
1528           NaN          NaN  Fri Oct 21 18:16:44 +0000 2016   
116            NaN          NaN  Sat Nov 21 17:05:31 +0000 2015   
1751           NaN          NaN  Fri Jan 27 00:24:48 +0000 2017   
634            NaN          NaN  Sun Dec 27 21:52:07 +0000 2015   
2301           NaN          NaN  Sun Jan 10 04:04:10 +0000 2016   
1452           NaN          NaN  Sat Sep 17 16:57:35 +0000 2016   
2202           NaN          NaN  Thu Sep 22 01:16:45 +0000 2016   
1188           NaN          NaN  Sun Jun 05 23:53:41 +0000 2016   
153            NaN          NaN  Mon Nov 23 04:59:42 +0000 2015   
2246           NaN          NaN  Mon May 23 01:40:38 +0000 2016   

                                               entities  \
1528  {'hashtags': [], 'symbols': [], 'user_mentions...   
116   {'hashtags': [], 'symbols': [], 'user_mentions...   
1751  {'hashtags': [], 'symbols': [], 'user_mentions...   
634   {'hashtags': [], 'symbols': [], 'user_mentions...   
2301  {'hashtags': [], 'symbols': [], 'user_mentions...   
1452  {'hashtags': [], 'symbols': [], 'user_mentions...   
2202  {'hashtags': [], 'symbols': [], 'user_mentions...   
1188  {'hashtags': [], 'symbols': [], 'user_mentions...   
153   {'hashtags': [], 'symbols': [], 'user_mentions...   
2246  {'hashtags': [], 'symbols': [], 'user_mentions...   

                                      extended_entities  favorite_count  \
1528  {'media': [{'id': 789530855911882752, 'id_str'...           12407   
116   {'media': [{'id': 668113012746776577, 'id_str'...             661   
1751                                                NaN           15572   
634   {'media': [{'id': 681231097963921409, 'id_str'...            2470   
2301                                                NaN            1189   
1452  {'media': [{'id': 777189753083092992, 'id_str'...           14968   
2202                                                NaN             903   
1188  {'media': [{'id': 739606132583456768, 'id_str'...            5517   
153   {'media': [{'id': 668655136865181697, 'id_str'...             529   
2246                                                NaN            1528   

      favorited  geo                  id              id_str  ...  \
1528      False  NaN  789530877013393408  789530877013393408  ...   
116       False  NaN  668113020489474048  668113020489474048  ...   
1751      False  NaN  824775126675836928  824775126675836928  ...   
634       False  NaN  681231109724700672  681231109724700672  ...   
2301      False  NaN  686035780142297088  686035780142297088  ...   
1452      False  NaN  777189768882946048  777189768882946048  ...   
2202      False  NaN  778764940568104960  778764940568104960  ...   
1188      False  NaN  739606147276148736  739606147276148736  ...   
153       False  NaN  668655139528511488  668655139528511488  ...   
2246      False  NaN  734559631394082816  734559631394082816  ...   

     quoted_status  quoted_status_id  quoted_status_id_str  retweet_count  \
1528           NaN               NaN                   NaN           3621   
116            NaN               NaN                   NaN            240   
1751           NaN               NaN                   NaN           3728   
634            NaN               NaN                   NaN            507   
2301           NaN               NaN                   NaN            126   
1452           NaN               NaN                   NaN           4582   
2202           NaN               NaN                   NaN            387   
1188           NaN               NaN                   NaN           1723   
153            NaN               NaN                   NaN            220   
2246           NaN               NaN                   NaN            413   

      retweeted  retweeted_status  \
1528      False               NaN   
116       False               NaN   
1751      False               NaN   
634       False               NaN   

In [137]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2334 entries, 0 to 2333
Data columns (total 30 columns):
contributors                     0 non-null float64
coordinates                      0 non-null float64
created_at                       2334 non-null object
entities                         2334 non-null object
extended_entities                1819 non-null object
favorite_count                   2334 non-null int64
favorited                        2334 non-null bool
geo                              0 non-null float64
id                               2334 non-null int64
id_str                           2334 non-null int64
in_reply_to_screen_name          77 non-null object
in_reply_to_status_id            77 non-null float64
in_reply_to_status_id_str        77 non-null float64
in_reply_to_user_id              77 non-null float64
in_reply_to_user_id_str          77 non-null float64
is_quote_status                  2334 non-null bool
lang                             2334 non-null o

In [138]:
for col in df_tweets.columns:
    print(df_tweets[col].value_counts().count(), '\t', col)

0 	 contributors
0 	 coordinates
2334 	 created_at
2240 	 entities
1819 	 extended_entities
1988 	 favorite_count
1 	 favorited
0 	 geo
2334 	 id
2334 	 id_str
30 	 in_reply_to_screen_name
76 	 in_reply_to_status_id
76 	 in_reply_to_status_id_str
30 	 in_reply_to_user_id
30 	 in_reply_to_user_id_str
2 	 is_quote_status
9 	 lang
1 	 place
1 	 possibly_sensitive
1 	 possibly_sensitive_appealable
24 	 quoted_status
26 	 quoted_status_id
26 	 quoted_status_id_str
1746 	 retweet_count
1 	 retweeted
165 	 retweeted_status
4 	 source
2334 	 text
2 	 truncated
89 	 user


In [140]:
df_tweets.possibly_sensitive.value_counts()

False    2200
Name: possibly_sensitive, dtype: int64

### Quality
#### `twitter_archive_enhanced` table
- There are "None" as value needs to be corrected.
- "name" column has anomalous names as one charactor name or start with small charactor.
- There are 4 columns, end with "_id", are float64 type while they need to be int64.
- There are 2 columns, end with "timestamp", are object type while they need to be time formate.
- "source" column has values in HTML fromat.
- There are 181 retweets.
- "rating_numerator" has anomalous values repeated 2 times or less.
- "rating_denominator" has values differ 10.
- There are 156 tweets as retweets.
- There are 44 tweets as replies.

#### `twitter_archive_enhanced` table
- there are 324 observations are not predected as a dog.

#### `tweets` table
- "created_at" is object type while it needs to be time form formate.
- There are columns should be droped becuase either they have 1 repeated value or nothing.

### Tidiness
- "doggo", "floofer", "pupper" & "puppo" columns of "twitter_archive_enhanced" have to be values in the new column called "type".
- all unwanted columns of "tweets" should be removed.

<a id='clean'></a>
<h2 id="-Gather" style="
    background-color: #555;
    color: #eee;
    padding: 10px 5px;
">Clean</h2>

> There are "None" as value needs to be corrected.

### Define

### Code

### Test